In [9]:
import firebase_admin
from firebase_admin import credentials, db, storage
from math import floor, sqrt
import ast
import smtplib
import plotly.express as px
import pandas as pd
import json
import streamlit as st
import plotly.graph_objects as go

In [2]:
faction_list = [
    "🧝 Unaffilated",
    "🏴 Blackthorne Company",
    "💰 Guild of the Black Sky",
    "🛡 Eponore",
    "⚜️ Catalpa",
    "🍷 Cedar Hill",
    "🧛‍♂️ The Dismissed",
    "💀 Geth",
    "❄️ Grimfrost",
    "🌳 The Grove",
    "🌙 The Irregulars",
    "⚖️ The Order",
    "🎪 Prismatic Troupe",
    "⚔️ Sunsteel Company",
    "🦁 Kult of Tharros",
    "🐴 Vidarian Khanate",
    "🏹 The Wardens",
    "🕊️ The White Ravens "
]

path_list = [
    '🗡 Warrior',
    '🪤 Rogue',
    '🩹 Healer',
    '🔮 Wizard'
]

def get_tier(events):
    return floor((sqrt(8*events)-1)/2)

if not firebase_admin._apps:
    key_dict = json.loads(st.secrets["firebase"], strict=False)
    creds = credentials.Certificate(key_dict)
    defualt_app = firebase_admin.initialize_app(creds, {
        'databaseURL': 'https://la-character-sheets-default-rtdb.firebaseio.com',
        'storageBucket':'la-character-sheets.appspot.com'
    })

In [82]:
user_data = db.reference("users/").get()
user_table = []
for key in user_data.keys():
    try:
        user_events = pd.DataFrame(json.loads(user_data[key]['event_info']))
        user_events.reset_index(drop=True, inplace=True)
        tier = get_tier(len(user_events[user_events['Event Type'] != "🪚 Work Weekend"]))
        skill_points = int(user_events["Skill Points"].sum())
        try:
            avail_points = int(user_events["Skill Points"].sum()) - int(user_data[key]['point_spend'])
        except:
            avail_points = skill_points
    except:
        skill_points = 0
        tier = 0
        avail_points = skill_points

    user_table.append({
        'Username':key,
        'Player':user_data[key]['name'],
        'Character':user_data[key]['character_name'],
        'Faction':user_data[key]['faction'],
        'Path':user_data[key]['path'],
        'Tier':tier,
        'Earned Points':skill_points,
        "Available Points":avail_points
    })
user_df = pd.DataFrame(user_table)

In [4]:
user_df

,Username,Player,Character,Faction,Path,Tier,Earned Points,Available Points
0,caenis_macneary,Niki Hunter,Caenis MacNeary,🎪 Prismatic Troupe,🗡 Warrior,4,28,0
1,ghostfox,Kara,Elora,🎪 Prismatic Troupe,🕳 Rogue,0,1,1
2,ntdens,Nate Densmore,Kython,🎪 Prismatic Troupe,🩸 Healer,5,43,11


In [10]:
tier_df = user_df.groupby('Tier').count()['Username']
tier_list = pd.DataFrame({'Tier':list(range(0,11))})
tier_df = tier_list.merge(tier_df, how='left', on='Tier').fillna(0).rename(columns={'Username':'Players'})
go.Figure(go.Bar(x=tier_df['Tier'], y=tier_df['Players']))

In [51]:
player_events = []
for player in user_df['Username']:
    try:
        user_events = pd.DataFrame(json.loads(user_data[player]['event_info']))
        user_events.reset_index(drop=True, inplace=True)
        user_events = user_events[user_events['Event Type'] != "🪚 Work Weekend"]
        try:
            user_events['Event Date'] = pd.to_datetime(user_events['Event Date'], format="%B %Y")
        except:
            pass
        try:
            user_events['Event Date'] = pd.to_datetime(user_events['Event Date'], unit='ms')
        except:
            pass
        player_events.append(pd.DataFrame({'Date':list(user_events['Event Date']),'Player':player}))
    except:
        pass

In [52]:
attend = pd.concat(player_events)
attend['Date'] = attend.Date - pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1)
attend = attend.groupby('Date').count().reset_index()
px.line(attend, x='Date', y='Player', title='Attendance Over Time')

In [74]:
bucket = storage.bucket()

In [75]:
blob = bucket.blob('ntdens')

ntdens/profile_pic.jpg
ntdens/profile_pic.png


In [83]:
image_location = user_data['ntdens']['pic_name']

In [84]:
image_location

'ntdens/profile_pic.png'

In [86]:
for b in bucket.list_blobs(prefix='ntdens'):
    if b.name != image_location:
        b.delete()

In [91]:
(st.secrets['admins'] + list(st.secrets['faction_leaders']))

['ntdens', 'caenis_macneary']